# Aprendizaje Estadístico y Data Mining

## Práctica 2: Patrones Secuenciales

### Objetivo
El conjunto de datos **“Navegación_Web.csv”** contiene clientes y las acciones que hace en una Web, teniendo estas asociadas un timestamp.

* [Link al dataset.](./data/Navegacion_Web.csv)

**Enunciado:** Habrá que procesar el fichero para crear una estructura de datos de los distintos clientes con las secuencias de las acciones realizadas en distintos instantes 

**Solución**


En primer lugar, realiremos un Dataframe a partir del csv indicado. De esta manera, podremos trabajar más comodamente con los datos,

In [184]:
import pandas as pd
df = pd.read_csv("resources/Navegacion_Web.csv")

df = df.sort_values(by=['UserID', 'Timestamp'])

print(df)


     UserID            Timestamp       Action
5    User_1  2024-01-01 12:46:08     Wishlist
1    User_1  2024-01-01 12:46:09         View
39   User_1  2024-01-01 12:46:10  Add to Cart
59   User_1  2024-01-01 12:46:16     Purchase
51   User_1  2024-01-01 12:46:18     Purchase
43   User_1  2024-01-01 12:46:19     Wishlist
3    User_1  2024-01-01 12:46:38     Purchase
45  User_10  2024-01-01 13:26:00      Contact
32  User_10  2024-01-01 13:26:02     Purchase
18  User_10  2024-01-01 13:26:12     Purchase
37  User_10  2024-01-01 13:26:13  Add to Cart
47  User_10  2024-01-01 13:26:14      Contact
26   User_2  2024-01-01 12:34:00      Contact
42   User_2  2024-01-01 12:34:02         View
33   User_2  2024-01-01 12:34:03         View
44   User_2  2024-01-01 12:34:29     Purchase
20   User_2  2024-01-01 12:34:30         View
9    User_3  2024-01-01 12:07:00         View
41   User_3  2024-01-01 12:07:23      Contact
27   User_3  2024-01-01 12:07:24      Contact
54   User_3  2024-01-01 12:07:26  

Ya con el Dataframe generado, procederemos con la creación de la estructura de datos. Para ello, tendremos en cuenta las transacciones realizadas por cada usuario en cada instante de tiempo, de tal manera que podamos averiguar la secuencia de acciones del mismo. La estructura que estaremos utilizando será un diccionario, cuya *key* sea el ID del usuario, y su valor una lista (que representará la secuencia) de listas (que representarán las transacciones).

In [185]:
secuencias_usuarios = df.groupby('UserID')['Action'].apply(list).to_dict()

print(secuencias_usuarios)

{'User_1': ['Wishlist', 'View', 'Add to Cart', 'Purchase', 'Purchase', 'Wishlist', 'Purchase'], 'User_10': ['Contact', 'Purchase', 'Purchase', 'Add to Cart', 'Contact'], 'User_2': ['Contact', 'View', 'View', 'Purchase', 'View'], 'User_3': ['View', 'Contact', 'Contact', 'Add to Cart', 'View', 'Logout'], 'User_4': ['Search', 'Add to Cart', 'Contact', 'Add to Cart', 'View', 'Contact'], 'User_5': ['Add to Cart', 'Purchase', 'View', 'Add to Cart', 'Wishlist', 'Contact'], 'User_6': ['Wishlist', 'View', 'Add to Cart', 'Purchase', 'View', 'Search', 'View'], 'User_7': ['View', 'Add to Cart', 'Contact', 'Add to Cart', 'Purchase', 'Logout'], 'User_8': ['Add to Cart', 'Add to Cart', 'Purchase', 'Purchase', 'Purchase', 'Wishlist'], 'User_9': ['Contact', 'Add to Cart', 'Purchase', 'Add to Cart', 'Purchase', 'Wishlist']}


Una vez hecho esto y analizando esta información se pueden extraer conclusiones sobre en qué orden compran los clientes los productos y así tomar decisiones de negocio. Este estudio se puede llevar a cabo aplicando el algoritmo Generalized Sequential Patterns utilizando la implementación de éste disponible en la librería *gsppy*.

**Enunciado:** Prueba al menos tres configuraciones de soporte diferentes. ¿Qué diferencias hay en los resultados y a que se debe?

**Solución**

En primer lugar, debemos transformar nuestro diccionario a un formato válido para gsppy. Para ello, realizaremos una lista apartir del diccionario.

In [186]:
from gsppy.gsp import GSP # type: ignore
import matplotlib.pyplot as plt

secuencias = [actions for actions in secuencias_usuarios.values()]
resultado1 = GSP(secuencias).search(min_support=0.2)

for i, patron in enumerate(resultado1):
    print(f"Patron de tamaño {i + 1}:")
    for patron, frecuencia in patron.items():
        patron_str = ' -> '.join(patron)
        print(f"  {patron_str}: {frecuencia} ocurrencias")
    print("\n")


Patron de tamaño 1:
  Wishlist: 5 ocurrencias
  View: 7 ocurrencias
  Add to Cart: 9 ocurrencias
  Purchase: 8 ocurrencias
  Contact: 7 ocurrencias
  Logout: 2 ocurrencias
  Search: 2 ocurrencias


Patron de tamaño 2:
  Wishlist -> View: 2 ocurrencias
  View -> Add to Cart: 4 ocurrencias
  View -> Contact: 2 ocurrencias
  Add to Cart -> View: 2 ocurrencias
  Add to Cart -> Purchase: 6 ocurrencias
  Add to Cart -> Contact: 3 ocurrencias
  Purchase -> Wishlist: 3 ocurrencias
  Purchase -> View: 3 ocurrencias
  Purchase -> Add to Cart: 2 ocurrencias
  Contact -> Add to Cart: 4 ocurrencias


Patron de tamaño 3:
  Wishlist -> View -> Add to Cart: 2 ocurrencias
  View -> Add to Cart -> Purchase: 2 ocurrencias
  Add to Cart -> Purchase -> View: 2 ocurrencias
  Add to Cart -> Contact -> Add to Cart: 2 ocurrencias
  Contact -> Add to Cart -> View: 2 ocurrencias
  Contact -> Add to Cart -> Purchase: 2 ocurrencias


Patron de tamaño 4:
  Wishlist -> View -> Add to Cart -> Purchase: 2 ocurrencias


Repetiremos este proceso, pero esta vez estableciendo el soporte a 0.3, de tal manera que sea un poco más estricto en cuanto a los valores que pasan y nos queden únicamente datos más relevantes en cuanto a frecuencia.

In [187]:
resultado2 = GSP(secuencias).search(min_support=0.3)

for i, patron in enumerate(resultado2):
    print(f"Patron de tamaño {i + 1}:")
    for patron, frecuencia in patron.items():
        patron_str = ' -> '.join(patron)
        print(f"  {patron_str}: {frecuencia} ocurrencias")
    print("\n")

Patron de tamaño 1:
  Wishlist: 5 ocurrencias
  View: 7 ocurrencias
  Add to Cart: 9 ocurrencias
  Purchase: 8 ocurrencias
  Contact: 7 ocurrencias


Patron de tamaño 2:
  View -> Add to Cart: 4 ocurrencias
  Add to Cart -> Purchase: 6 ocurrencias
  Add to Cart -> Contact: 3 ocurrencias
  Purchase -> Wishlist: 3 ocurrencias
  Purchase -> View: 3 ocurrencias
  Contact -> Add to Cart: 4 ocurrencias




Por último, ampliaremos un poco más el soporte a 0.4, para ver si limpamos más aún frecuencias irrelevantes.

In [188]:
resultado3 = GSP(secuencias).search(min_support=0.4)

for i, patron in enumerate(resultado3):
    print(f"Patron de tamaño {i + 1}:")
    for patron, frecuencia in patron.items():
        patron_str = ' -> '.join(patron)
        print(f"  {patron_str}: {frecuencia} ocurrencias")
    print("\n")

Patron de tamaño 1:
  Wishlist: 5 ocurrencias
  View: 7 ocurrencias
  Add to Cart: 9 ocurrencias
  Purchase: 8 ocurrencias
  Contact: 7 ocurrencias


Patron de tamaño 2:
  View -> Add to Cart: 4 ocurrencias
  Add to Cart -> Purchase: 6 ocurrencias
  Contact -> Add to Cart: 4 ocurrencias




**Enunciado:** Para una de ellas, interpreta algunos de los patrones secuenciales que te resulten curiosos.

**Solución**

Interpretaremos los patrones del primero de los estudios, el de soporte = 0.2, ya que es el único que tiene patrones de tamaño superior a 2, lo que hace que podamos encontrar secuencias más interesantes.  

En primer lugar, repetiremos el código, pero esta vez sin mostrar los patrones de tamaño 1, ya que no nos interesa.

In [ ]:
for i, patron in enumerate(resultado1):
    if i + 1 >= 2:  # Solo imprime patrones de tamaño 2 en adelante
        print(f"Patron de tamaño {i + 1}:")
        for secuencia, frecuencia in patron.items():
            patron_str = ' -> '.join(secuencia)
            print(f"  {patron_str}: {frecuencia} ocurrencias")
        print("\n")

**Enunciado:** ¿Qué transacción es clave eliminar para cambiar los patrones obtenidos en el punto anterior? ¿Y cuál de las secuencias?

**Solución**